In [4]:
import json
import numpy as np
from skimage.io import imread
from skimage.transform import resize
from skimage.color import gray2rgb
import boto3
from io import BytesIO


In [5]:
# Load configuration file
with open('config.json') as config_file:
    config = json.load(config_file)

# Extract configuration parameters
s3_bucket_name = config["s3_bucket_name"]
img_rows = config["img_rows"]
img_cols = config["img_cols"]
as_gray = config["as_gray"]
channels = config["in_channel"]
s3_train_path_prefix = config["s3_train_path_prefix"]
s3_test_path_prefix = config["s3_test_path_prefix"]

# Initialize S3 client
s3 = boto3.client('s3')


In [6]:
def read_image_from_s3(s3_key, as_gray=True):
    """Reads an image from S3 and returns it as a NumPy array."""
    obj = s3.get_object(Bucket=s3_bucket_name, Key=s3_key)
    image = imread(BytesIO(obj['Body'].read()), as_gray=as_gray)
    return image

In [9]:
import numpy as np
import boto3
from skimage.io import imread
from skimage.transform import resize
from skimage.color import gray2rgb
from io import BytesIO

# Initialize S3 client
s3 = boto3.client('s3')
bucket_name = config["s3_bucket_name"]

# Define the function to read and combine left and right images for each view
def read_and_combine_views(left_paths, right_paths, img_rows, img_cols, as_gray, channels, bucket_name, prefix):
    images_combined = []
    for left_path, right_path in zip(left_paths, right_paths):
        try:
            # Load left image
            left_s3_key = f"{prefix}/{left_path}"
            left_obj = s3.get_object(Bucket=bucket_name, Key=left_s3_key)
            left_image = imread(BytesIO(left_obj['Body'].read()), as_gray=as_gray)
            if as_gray and channels == 3:
                left_image = gray2rgb(left_image)
            left_image = resize(left_image, (img_rows, img_cols), anti_aliasing=True)

            # Load right image
            right_s3_key = f"{prefix}/{right_path}"
            right_obj = s3.get_object(Bucket=bucket_name, Key=right_s3_key)
            right_image = imread(BytesIO(right_obj['Body'].read()), as_gray=as_gray)
            if as_gray and channels == 3:
                right_image = gray2rgb(right_image)
            right_image = resize(right_image, (img_rows, img_cols), anti_aliasing=True)

            # Stack left and right images along the last dimension
            combined_image = np.stack((left_image, right_image), axis=-1)  # Shape: (img_rows, img_cols, channels * 2)
            images_combined.append(combined_image)
        except Exception as e:
            print(f"Error loading {left_path} or {right_path}: {e}")
    
    # Convert to numpy array and normalize
    images_combined = np.asarray(images_combined, dtype=np.float32)
    images_combined = (images_combined - images_combined.min()) / (images_combined.max() - images_combined.min())
    return images_combined


In [7]:
def read_mg(file_paths, img_rows, img_cols, as_gray, channels, bucket_name, prefix):
    images = []
    s3 = boto3.client('s3')  # Reinitialize in case of multi-threading issues
    for file_path in file_paths:
        # Construct S3 key for the image
        s3_key = f"{prefix}/{file_path}"
        try:
            # Load image from S3
            obj = s3.get_object(Bucket=bucket_name, Key=s3_key)
            image = imread(BytesIO(obj['Body'].read()), as_gray=as_gray)
            
            # Convert grayscale to RGB if necessary
            if as_gray and channels == 3:
                image = gray2rgb(image)
            
            # Resize image
            image = resize(image, (img_rows, img_cols), anti_aliasing=True)
            images.append(image)
        except Exception as e:
            print(f"Error loading {s3_key}: {e}")
    
    # Convert to numpy array and normalize
    images = np.asarray(images, dtype=np.float32)